In [ ]:
from dataclasses import dataclass, fields, field
from typing import List, Dict, Optional, get_origin, Union, Any
from enum import Enum

class WarningSeverity(Enum):
    INFO = "INFO"
    WARNING = "WARNING"
    ERROR = "ERROR"

class UnresolvedVaultErrors(Exception):
    def __init__(self, count):
        plural = "s" if count != 1 else ""
        message = (
            f"Vault construction failed: {count} validation error{plural} detected.\n"
            f"Please review the validation summary above and resolve all errors.\n"
            f"Tip: Run with execute=False to preview queries without validation blocking."
        )
        super().__init__(message)

class ValidateEmptyParams:
    def __post_init__(self):
        for f in fields(self):
            value = getattr(self, f.name)
            
            is_optional = get_origin(f.type) is Union and type(None) in f.type.__args__
            if not is_optional and (value is None or value == ""):
                if value is None or value == "":
                    raise ValueError(f"Field '{f.name}' in {self.__class__.__name__} cannot be empty.")
                if isinstance(value, list) and len(value) == 0:
                    raise ValueError(f"Field '{f.name}' in {self.__class__.__name__} cannot be an empty list.")

@dataclass
class ValidationIssue:
    severity: WarningSeverity
    stage: str
    entity: str
    message: str
    details: Optional[Dict] = None

@dataclass
class VaultEntityMetadata:
    schema: str
    table: str
    columns: List[str]

@dataclass
class Hub(ValidateEmptyParams):
    name: str
    schema_name: str
    business_key_columns: List[str]
    source_table: str
    load_datetime_column: Optional[str] = None
    _stage: Optional[VaultEntityMetadata] = field(init=False, repr=False, default=None)

    @property
    def hash_key_name(self) -> str:
        return f"hk_{self.name.replace('hub_', '')}"

    @property
    def stage(self) -> VaultEntityMetadata:
        if self._stage is None:
            raise RuntimeError("stage must be set before access")
        return self._stage

    def _set_stage(self, ctx: VaultEntityMetadata) -> None:
        """Internal use only."""
        self._stage = ctx

@dataclass
class RegisteredHub:
    name: str
    schema_name: str
    business_key_columns: List[str]

    @property
    def hash_key_name(self) -> str:
        return f"hk_{self.name.replace('hub_', '')}"

@dataclass
class LinkAnchor:
    table: str
    hub: Optional[str] = None
    bk_columns: List[str] = field(default_factory=list)

@dataclass
class LinkHubJoin:
    hub: str
    table: str
    bk_columns: List[str]                    # Column(s) to hash for this hub's HK
    join_on: Optional[Dict[str, str]] = None  # {"fk_col": "pk_col"}

@dataclass
class Link(ValidateEmptyParams):
    name: str
    schema_name: str
    staging_schema: str
    anchor: LinkAnchor
    hub_mapping: List[LinkHubJoin]
    load_datetime_column: Optional[str] = None
    source_columns: Optional[List[str]] = None # auto-detect hub via columns, fragile if there are multiple hubs with same column
    _stage: Optional[VaultEntityMetadata] = field(init=False, repr=False, default=None)

    @property
    def hash_key_name(self) -> str:
        return f"hk_{self.name.replace('link_', '')}"

    @property
    def stage(self) -> VaultEntityMetadata:
        if self._stage is None:
            raise RuntimeError("stage must be set before access")
        return self._stage

    def _set_stage(self, ctx: VaultEntityMetadata) -> None:
        """Internal use only."""
        self._stage = ctx

@dataclass
class RegisteredLink:
    name: str
    schema_name: str

    @property
    def hash_key_name(self) -> str:
        return f"hk_{self.name.replace('link_', '')}"

@dataclass
class Satellite(ValidateEmptyParams):
    name: str
    schema_name: str
    parent_hub_or_link: str
    descriptive_columns: List[str]
    source_table: str
    include_mode: bool = True
    hash_column: Optional[List[str]] = None
    load_datetime_column: Optional[str] = None
    _stage: Optional[VaultEntityMetadata] = field(init=False, repr=False, default=None)

    @property
    def stage(self) -> VaultEntityMetadata:
        if self._stage is None:
            raise RuntimeError("stage must be set before access")
        return self._stage

    def _set_stage(self, ctx: VaultEntityMetadata) -> None:
        """Internal use only."""
        self._stage = ctx

    @property
    def resolved_columns(self) -> List[str]:
        if self._resolved_columns is None:
            raise RuntimeError("resolved_columns must be set before access. Run __process_satellites() first.")
        return self._resolved_columns

    def _set_resolved_columns(self, cols: List[str]) -> None:
        self._resolved_columns = cols

@dataclass
class Stage:
    source_table: str
    name: str = ""
    schema_name: str = "staging"
    use_default_naming_scheme: bool = True

    hash_keys: Dict[str, List[str]] = field(default_factory=dict)     # from Hub, Link
    hash_diffs: Dict[str, List[str]] = field(default_factory=dict)    # from satellite

    def __post_init__(self):
        if self.use_default_naming_scheme and not self.name:
            table_name = self.source_table.split(".")[-1]
            self.name = f"stg_{table_name}"